### Define your own acquisition function

https://nbviewer.jupyter.org/github/SheffieldML/GPyOpt/blob/devel/manual/GPyOpt_creating_new_aquisitions.ipynb

GPyOpt BO Tutorial: https://www.blopig.com/blog/wp-content/uploads/2019/10/GPyOpt-Tutorial1.html

GpyOpt Github page: https://github.com/SheffieldML/GPyOpt

Example 1: https://github.com/SheffieldML/GPyOpt/blob/master/manual/GPyOpt_bandits_optimization.ipynb

Example 2: https://nbviewer.jupyter.org/github/SheffieldML/GPyOpt/blob/master/manual/GPyOpt_reference_manual.ipynb

In [1]:
from GPyOpt.acquisitions.base import AcquisitionBase
from GPyOpt.core.task.cost import constant_cost_withGradients
from scipy.stats import norm
from scipy import integrate
import numpy as np
    
class AcquisitionNew_EI(AcquisitionBase):
    
    """
    General template to create a new GPyOPt acquisition function

    :param model: GPyOpt class of model
    :param space: GPyOpt class of domain
    :param optimizer: optimizer of the acquisition. Should be a GPyOpt optimizer
    :param cost_withGradients: function that provides the evaluation cost and its gradients

    """

    # --- Set this line to true if analytical gradients are available
    analytical_gradient_prediction = False

    
    def __init__(self, model, space, optimizer, cost_withGradients=None, **kwargs):
        self.optimizer = optimizer
        super(AcquisitionNew_EI, self).__init__(model, space, optimizer)
        
        # --- UNCOMMENT ONE OF THE TWO NEXT BITS
             
        # 1) THIS ONE IF THE EVALUATION COSTS MAKES SENSE
        #
        # if cost_withGradients == None:
        #     self.cost_withGradients = constant_cost_withGradients
        # else:
        #     self.cost_withGradients = cost_withGradients

        # 2) THIS ONE IF THE EVALUATION COSTS DOES NOT MAKE SENSE
        #
        if cost_withGradients == None:
            self.cost_withGradients = constant_cost_withGradients
        else:
            print('The new acquisition does now make sense with cost. Cost set to constant.')
            self.cost_withGradients = constant_cost_withGradients


    def _compute_acq(self,x):
        
        # --- DEFINE YOUR AQUISITION HERE (TO BE MAXIMIZED)
        #
        # Compute here the value of the new acquisition function. Remember that x is a 2D  numpy array
        # with a point in the domanin in each row. f_acqu_x should be a column vector containing the
        # values of the acquisition at x.
        #
        m_vec, s2_vec = self.model.predict(x)
        
        # target a, alpha should be inputs (to move to __init__)
        a = 0
        alpha = 0.1
        
        f_acqu_x = []
        for index in range(len(m_vec)):
            m = m_vec[index]; s2 = s2_vec[index]; s = np.sqrt(s2)
            f = lambda y: (y-m)**2 * norm.pdf((y-m)/s) # integrand for the third term
            f_acqu_x_temp = ((alpha**2 * s2) - (m-a)**2) * (norm.cdf((a-m)/s+alpha) - norm.cdf((a-m)/s-alpha)) \
                    + 2*(m-a)*s2 * (norm.pdf((a-m)/s+alpha) - norm.pdf((a-m)/s-alpha)) \
                    + integrate.quad(f, a-alpha*s, a+alpha*s)[0]
            f_acqu_x.append(f_acqu_x_temp)
        
        f_acqu_x = np.array(f_acqu_x); f_acqu_x = f_acqu_x.reshape(-1,1)

        return f_acqu_x
    
    def _compute_acq_withGradients(self, x):
        
        # --- DEFINE YOUR AQUISITION (TO BE MAXIMIZED) AND ITS GRADIENT HERE HERE
        #
        # Compute here the value of the new acquisition function. Remember that x is a 2D  numpy array
        # with a point in the domanin in each row. f_acqu_x should be a column vector containing the
        # values of the acquisition at x. df_acqu_x contains is each row the values of the gradient of the
        # acquisition at each point of x.
        #
        # NOTE: this function is optional. If note available the gradients will be approxiamted numerically.
        
        return f_acqu_x, df_acqu_x
